# Data Augmentation
___
by Oliver Thomaschewski (Matrikelnummer) and Lisa-Marlen Wiegandt (572770)

## Table of content
___
1. [What is data augmentation](#What-is-data-augmentation)
1. [Why data augmentation](#Why-data-augmentation)
    2. Challenges
    2. Use Cases
1. [Usefulness of data augmentation](#Usefulness-of-augmentation)
1. [Example: Image augmentation](#Image-augmentation)
    2. Simple transformations
    2. Advanced techniques
1. [Sources](#Sources)

## What is data augmentation
___
* artificially increase of data set
* synthesizing new data from given data
    * amount of data increased
    * diversity of data increased


## Why data augmentation
___
* prediction accuracy based on amount and diversity of training data
* therefore imporving prediction (best case)
* reducing cost of collecting and labeling data
* preventing data privacy problems
    
#### Challenges
* useful augmentation
* needs to analyze output quality of augmentation
* reproducing biases from original dataset  

#### Use Cases

* Small dataset for medical images, especially for rare diseases
* due to data privacy regulations data is not given

## Usefulness of data augmentation
___
* comparison augmented dataset with no agmented dataset via base line model
* Image classification without augmentation useless

> Reminder: a baseline model uses heuristics, simple summary statistics, randomness, or machine learning for predictions

In [1]:
import numpy as np
from keras.utils import np_utils
from keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(X_test[i]) 
    plt.axis('off')
plt.show()
print('Labels: %s' %(y_test[0:5]))

#https://analyticsindiamag.com/image-data-augmentation-impacts-performance-of-image-classification-with-codes/

ModuleNotFoundError: No module named 'tensorflow'

## Image augmentation
___
* simple alteration popular 
* or generation of new synthetic data with more advanced techniques

Some examples of data augmentation with torchvision.transforms

we start with one picture ##maybe htw picture ??????

In [ ]:
from PIL import Image
import PIL
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import sys
import torch
import numpy as np
import torchvision.transforms as T

### Simple transformation

Image to Gray Scale

In [ ]:
orig_img = Image.open(Path("images\pexels-eva-bronzini-5502308.jpg"))
gray_img = T.Grayscale()(orig_img)
gray_img.save("images\img_gray.jpg")

### Advanced techniques

Add Gaussian Blur

In [ ]:
blurred_imgs = [T.GaussianBlur(kernel_size=(51, 91), sigma=sigma)(orig_img) for sigma in (3,7)]
blurred_imgs.save("images\img_blur.jpg")

## Sources

first 
https://www.mygreatlearning.com/blog/understanding-data-augmentation/
https://research.aimultiple.com/data-augmentation/

second
https://snapstack.cz/data-augmentation-advantages-challenges-and-instances/
https://research.aimultiple.com/data-augmentation/

third
https://towardsdatascience.com/augmentation-for-image-classification-24ffcbc38833

fourth
https://towardsdatascience.com/a-comprehensive-guide-to-image-augmentation-using-pytorch-fb162f2444be